## Modified approach

- INSTRUCTs have the desired intended effect of changing the state of the world
    - what is the speaker's desired ontic intended effect. What state of the world would the speaker like the listener to make happen 
- STATEMENTs have the desired intended effect of changing the state of the listener's beliefs
    - what is the speaker's desired epistemic effect on the listener. What state of the listener's mind would the speaker like the listener to make happen 
- QUESTIONs have the desired intended effect of changing the state of the speaker's beliefs 
    - what is the speaker's desired epistemic effect on themselves. What state of the speaker's mind would the speaker like the listener to help make happen.
    
    
We convert everything to a "goal" and have a planner figure out what needs to happen. We don't care about the action repertoire at all, unless there is ambiguity 

### Challenges 

- Issues with types and type hierarchy (is "container" a semantic type or is it a consultant property)
- Issues with action parameters (arbitrarily defined, not tied to semantic types, sometimes includes properties) 
- Novelty in the CPC vs novelty in the SPC. 
- agent as a physobj vs agent as a location.

## IMPORTS

In [6]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI,ChatAnthropic
from langchain.chains import LLMChain
import json

## Helper functions

In [4]:
# Given a list of dictionaries, and a key, return the entry in the list that matches
def find_dict_in_list(lst, key, target):
    for item in lst:
        if not key in item:
            #print("Key not in Dict")
            return None
        if item[key] == target:
            return item
    #print("Nothing found")
    return None

def find_all_dicts_in_list(lst, key, target):
    output = []
    for item in lst:
        if not key in item:
            return output
        if item[key] == target:
            output.append(item)
    #print("Nothing found")
    return output
    

def clean_candidates(candidates):
    """
    Cleans the list of candidates to extract a list of names and a list of scores of the candidates
    """
    names = []
    scores = []
    for candidate in candidates:
        name = candidate.split(":")[0]
        score = float(candidate.split(":")[1])
        names.append(name)
        scores.append(score)
        
    return names, scores


def prune_candidates(names, scores, threshold=0.75):
    return [(x,y) for x,y in zip(names,scores) if y > threshold ]

def select_best_candidate(names, scores, threshold=0.75):
    """
    Selects best name and score above a threshold. 
    """
    pruned_names = []
    pruned_scores = []
    for n,s in zip(names,scores):
        if s>threshold:
            pruned_names.append(n)
            pruned_scores.append(s)
    
    if pruned_names:
        return pruned_names[pruned_scores.index(max(pruned_scores))]
    return "NONE"

In [5]:
# More helper functions GMR

def central_referent(gmr):
    return find_dict_in_list(gmr['referents'], "role", "central")

def supp_referents(gmr):
    return find_all_dicts_in_list(gmr['referents'], "role", "supplemental")

## Initialize LLM

In [7]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
#llm = OpenAI(temperature=0.0)
#llm = Anthropic(model="claude-instant-1.1-100k")

## Chains

In [8]:
## (1) Speech Act Classification 

template_speech_act= """
Decide whether the utterance below from a speaker to a listener is one of "want", "wantBel", "itk"
A "want" is an imperative statement or a request by the speaker to have the listener do an action or stop doing an action.
An "itk" is a 'wh' or 'yes/no' query (what, why, when, where, who) or request from a speaker for more information from the listener about the listeners knowledge, beliefs or perceptions
A "wantBel" (note the uppercase B) is a statement of fact or opinion that the speaker conveys to a listener and  expects to listener to come to believe. 



utterance: \n{utterance}\n
act:
"""

prompt_speech_act = PromptTemplate(
    input_variables=["utterance"],
    template=template_speech_act
)

chain_speech_act = LLMChain(llm=llm, prompt=prompt_speech_act)

In [9]:
## (2) Central Referents 

template_centralref = """
What is the central entity or thing (which could be a single thing or a reference to a set) that is being referred to in the below sentence?

Remember, the central referent is the one single thing or object, not an action or descriptor. It is meant to capture the central real world or hypothesized item being referenced in the utterance. 


sentence: \n{utterance}\n 
referent:
"""

prompt_centralref = PromptTemplate(
    input_variables=["utterance"],
    template=template_centralref
)

chain_centralref = LLMChain(llm=llm, prompt=prompt_centralref)

In [10]:
## (3) Supporting Referents

template_suppref = """
What are some objects (which could be a single thing or a collection of things) that is being referred to in the below sentence not including the central referent? Return as a python list.
If none, then return empty list []. Even if only one item, return as a list.  
Remember, the supporting referents are things or objects, not actions or descriptors. They are meant to capture the real world items being referenced in the utterance. 

Do NOT include objects or collections that have already been covered in the central referent. 

sentence: \n{utterance}\n 
central referent: \n{centralref}\n
supporting referents (noun(s) from utterance):
"""

prompt_suppref = PromptTemplate(
    input_variables=["utterance", "centralref"],
    template=template_suppref
)

chain_suppref = LLMChain(llm=llm, prompt=prompt_suppref)

In [11]:
## (4) Getting the type of thing that the referents are 

template_typeof = """
Determine whether or not the referent item mentioned below in the context of the provided utterance is one of the types also provided below. To check if the referent is of a type, follow the below procedure
1. Iterate through each item mentioned in the list of types. 
2. For each item X in the list of types expand on the meaning of each item in context of the utterance, and then ask if the central referent is of type X given that meaning. 
3. If the central referent is of type X in the list, return X.

\n\n EXAMPLE \n
utterance: The lemon is on the table
referent: lemon
types: ['area', 'physobj', 'location', 'pose']
typeOf: Looking through the items in the list of types above, physobj is a physical object. lemon is a type of physical object and its use in the utterance is that of a physobj. So, it is of type physobj

Remember, return specifically ONE of the items in the list, or if none apply then return NONE. 

utterance: \n{utterance}\n
referent: \n{ref}\n
types: \n{types}\n
typeOf:
"""

prompt_typeof = PromptTemplate(
    input_variables=["ref", "types", "utterance"],
    template=template_typeof
)

chain_typeof = LLMChain(llm=llm, prompt=prompt_typeof)